## Lats and Lons Plot on CartoPy Map 

This notebook stores the lats and lons of a float instrument's time series in a csv file and plots them on a map using CartoPy.

You can find the CartoPy projection list on the following link:
https://scitools.org.uk/cartopy/docs/latest/crs/projections.html#cartopy-projections

In [2]:
# import libraries
from datetime import datetime, timedelta
import numpy as np
import xarray
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
from pandas import Series, DataFrame
import gsw
import cmocean
import sys
import glob
import errno
import os
import os.path
import netCDF4 as nc
import geopandas as gpd
from cartopy import crs as ccrs 
import cartopy.feature as cfeat
import string
import time
import csv
import geopandas as gpd
from cartopy import crs as ccrs 
# or import cartopy.crs as ccrs
import cartopy.feature as cfeat

C:\Users\maria\AppData\Local\Continuum\anaconda3\lib\site-packages\cmocean\tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [3]:
datadir = '/path/datafolder/' # Insert folder path
for filename in os.listdir(datadir):
    #savedir = '/path/savefolder/'
    # Insert directory you'd like to save in
        
    #def filecount(dir_name):
    #   """This function counts number of files in directory provided"""
    #   return len([f for f in os.listdir(dir_name) if os.path.isfile(f)])
    fname = filename
        
    ncdata = xarray.open_dataset(datadir + fname)
    juld = pd.to_datetime(ncdata.time.values) # convert time values to datetime
    temp = ncdata.Temperature.values # in situ temperature
    press = ncdata.Pressure.values  # pressure (needed for depth)
    lats = np.array(ncdata.lat.values) # latitude
    lons = np.array(ncdata.lon.values) # longitude
    z = gsw.conversions.z_from_p(press, lats) # depth calculated from pressure
    maxtemp = temp.max() # maximum temperature
    a = temp.argmax() # index of the max temp
    depth = z[a] # depth at maximum temperature
        
    if maxtemp > 10 or depth < -700:
        continue
            
    latitude = ([lats])
    longitude = ([lons])
    data = {'Lat': latitude, 'Lon': longitude} 
    df = DataFrame(data, index=[juld]) # store lats and lons in a dataframe
    #print(df)
    
    with open("path\csvfile", "a")  as f:
        # Insert path for the file you'll save the dataframe in
        table1 = df.to_csv(f, sep=',', header=False)            

In [ ]:
ncdata

In [ ]:
df.head() # Check dataframe headers

In [ ]:
df1 = pd.read_csv('path\csvfile', header=None, error_bad_lines=False, delimiter=',')
# Insert file path you stored the attributes in
df1.columns = ['YearTime', 'latitude', 'longitude']
# Setup the column headers
df1

In [ ]:
# Plot plain projection
fig1 = plt.figure(figsize=(10, 8))

ax = fig1.add_subplot(1, 1, 1, projection=ccrs.Miller())
ax.add_feature(cfeat.LAND)
ax.add_feature(cfeat.OCEAN)
ax.add_feature(cfeat.COASTLINE)
ax.add_feature(cfeat.BORDERS, linestyle=':')
ax.add_feature(cfeat.LAKES, alpha=.5) 
# alpha indicates transperancy, so it would look different from the ocean
ax.add_feature(cfeat.RIVERS)

ax.set_extent([-150, 30, -90, 0])

In [ ]:
df2 = df1.set_index("YearTime", drop = False)

x=df2['latitude']
y=df2['longitude']

ax.scatter(y, x, transform=ccrs.Miller(), marker= '*', s=400, c='y') 
# Plot lats and lons on map

fig1

In [ ]:
# Plot on Stereographic projection
proj = ccrs.Stereographic(central_longitude=-70, central_latitude=-30) # map not covering this part of the globe!
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1, 1, 1, projection=proj)
ax.add_feature(cfeat.STATES.with_scale('50m'), edgecolor = 'black')
ax.add_feature(cfeat.BORDERS)
ax.gridlines()
ax.set_extent([-150, 30, -90, 0])
ax.scatter(y, x, transform=ccrs.PlateCarree(), marker= '*', s=400, c='y')